In [ ]:
'''
--------------------------------
Autor : Lozada Sánchez Alan Omar
Date : 02/12/2021
--------------------------------
Requirements :
adjustText==0.7.3
argon2-cffi==21.1.0
attrs==21.2.0
backcall==0.2.0
bioinfokit==2.0.6
bleach==4.1.0
certifi==2021.10.8
cffi==1.15.0
charset-normalizer==2.0.7
colorlover==0.3.0
combat==0.3.0
cufflinks==0.17.3
cycler==0.11.0
debugpy==1.5.1
decorator==5.1.0
defusedxml==0.7.1
entrypoints==0.3
GEOparse==2.0.3
gtfparse==1.2.1
idna==3.3
importlib-resources==5.4.0
ipykernel==6.5.0
ipython==7.29.0
ipython-genutils==0.2.0
ipywidgets==7.6.5
jedi==0.18.0
Jinja2==3.0.2
joblib==1.1.0
jsonschema==4.2.1
jupyter-client==7.0.6
jupyter-core==4.9.1
jupyterlab-pygments==0.1.2
jupyterlab-widgets==1.0.2
kaleido==0.2.1
kiwisolver==1.3.2
MarkupSafe==2.0.1
matplotlib==3.4.3
matplotlib-inline==0.1.3
matplotlib-venn==0.11.6
mistune==0.8.4
mpmath==1.1.0
nbclient==0.5.4
nbconvert==6.2.0
nbformat==5.1.3
nest-asyncio==1.5.1
notebook==6.4.5
numpy==1.18.5
packaging==21.2
pandas==1.1.5
pandocfilters==1.5.0
parso==0.8.2
patsy==0.5.1
pexpect==4.8.0
pickleshare==0.7.5
Pillow==8.4.0
plotly==5.3.1
prometheus-client==0.12.0
prompt-toolkit==3.0.22
ptyprocess==0.7.0
pycparser==2.20
Pygments==2.10.0
pyparsing==2.4.7
PyPDF2==1.26.0
pyrsistent==0.18.0
python-dateutil==2.8.2
pytz==2021.3
pyzmq==22.3.0
requests==2.26.0
scikit-learn==1.0.1
scipy==1.7.2
seaborn==0.11.2
Send2Trash==1.8.0
six==1.16.0
statsmodels==0.13.0
tabulate==0.8.9
tenacity==8.0.1
terminado==0.12.1
testpath==0.5.0
textwrap3==0.9.2
threadpoolctl==3.0.0
tornado==6.1
tqdm==4.62.3
traitlets==5.1.1
urllib3==1.26.7
wcwidth==0.2.5
webencodings==0.5.1
widgetsnbextension==3.5.2
zipp==3.6.0
'''

# Imports

In [1]:
import pandas as pd
import numpy as np
import re
import gzip
import shutil
import os
import io
import cufflinks as cf
import GEOparse
from pysradb.sraweb import SRAweb
from bioinfokit.analys import norm
from gtfparse import read_gtf
import matplotlib.pyplot as plt
from subprocess import call
from PyPDF2 import PdfFileMerger
from combat.pycombat import pycombat
import logging
import scipy.stats as stats
import warnings

C:\Python39\lib\site-packages\pysradb\utils.py:14: TqdmExperimentalWarning:

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)



# Configuraciones

In [ ]:
# Configuraciones para matplot
plt.rcParams["figure.figsize"] = (15,10)
# Configuraciones para pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.min_rows', 20)
# Configuraciones para cufflinks
cf.go_offline()
# Configuraciones para logging
# (mensajes de INFO, DEBUG, WARNING, ERROR, CRITICAL)
logger = logging.getLogger('APP')
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter(fmt='[%(name)s] %(asctime)s %(levelname)s - %(message)s',datefmt='%d-%m-%y %H:%M:%S')
_handler = logging.StreamHandler()
_handler.setLevel(logging.DEBUG)
_handler.setFormatter(formatter)
logger.handlers.clear() # Borra los handlers anteriores 
logger.addHandler(_handler)
# handler para escribir los mensajes en un archivo
# file_handler = logging.FileHandler('logs.log')
# file_handler.setLevel(logging.DEBUG)
# file_handler.setFormatter(formatter)
# logger.addHandler(file_handler)

# Establecer espacio y forma de trabajo

In [ ]:
# True si se va a trabajar en Linux, false si se trabaja en Windows.
isLinux = False
# True si se desea ver las graficas en pantalla.
# False si se va a ejecutar en linea de comandos (sin interfaz gráfica).
showGraphs = True
# Plataforma sobre la que se va a trabajar.
platform = 'GPL19081'

# Directorio de trabajo principal.
work_dir = f'G:/Trabajos/servicioSocial/{platform}'
os.makedirs(work_dir) if not os.path.exists(work_dir) else None # Crea la carpeta si no existe.
os.chdir (work_dir)
# Directorio para archivos soft (metadata).
metadata_dir = f'{work_dir}/metadata'
# Directorio para los experimentos (series).
experiments_dir = f'{work_dir}/experiments'
# Directorio para documentos (pdfs, csv).
documents_dir = f'{work_dir}/documents'

# Directorio del script de R (gene_count.R) para contar genes.
genecount_script = f'{work_dir}/gene_count.R'
# Directorio del archivo de referencia del genoma (.fna.gz).
genomaref_fnafile = f'{work_dir}/GCF_000149245.1_CNA3_genomic.fna.gz'
# Directorio del archivo de anotaciones del genoma (.gtf.gz).
genomaannot_gtffile = f'{work_dir}/GCF_000149245.1_CNA3_genomic.gtf.gz'
# Correo para descargas.
correo = 'correo@gmail.com'

###############################################################
### En Linux es necesario instalar sratoolkit y R.          ###
### En Windows se especifica la carpeta bin de sratoolkit y ###
### el directorio donde esta instalado R.                   ###
###############################################################
if isLinux:
    # Forma de ejecutar un script de R.
    rscript_dir = 'Rscript'
else:
    # Directorio bin de sratoolkit.
    sratoolkit_dir = f'{work_dir}/sratoolkit.2.11.1-win64/bin'
    # Directorio de R (Rscript.exe) para ejecutar archivos R.
    rscript_dir = 'C:/"Program Files"/R/R-4.1.1/bin/Rscript.exe'


In [ ]:
# Mensajes en pantalla de los directorios elegidos
#-------------------------------------------------
logger.info('Ejecución esblecida para Linux') if isLinux else logger.info('Ejecución establecida para Windows')
logger.info(f'Plataforma {platform}')
logger.info(f'Directorio principal {work_dir}')
logger.info(f'Directorio para metadata {metadata_dir}')
logger.info(f'Directorio para las series {experiments_dir}')
logger.info(f'Directorio para documentos {documents_dir}')
logger.info(f'Script de R para el conteo de genes {genecount_script}')
logger.info(f'Archivo de referencia del genoma {genomaref_fnafile}')
logger.info(f'Archivo de anotaciones del genoma {genomaannot_gtffile}')
logger.info(f'Correo electronico {correo}')
logger.info(f'Directorio de sratoolkit {sratoolkit_dir}') if not isLinux else None
logger.info(f'Directorio de R {rscript_dir}') if not isLinux else None
#-------------------------------------------------

# Descarga de metadata

In [ ]:
logger.info(f'Descarga de metadata para la plataforma {platform}')
# Descarga de metadata de la plataforma.
gpl = GEOparse.get_GEO(geo = platform, destdir = metadata_dir)
# Nombres de las series de la plataforma.
series = dict.fromkeys(gpl.metadata['series_id'], None)
logger.info(f'Descarga de metadata para la plataforma {platform} finalizada')

In [ ]:
logger.info(f'Descarga de metadata para las series de la plataforma {platform}')
# Descarga de metadata de las series
for serie in series.keys():
    series[ serie ] = GEOparse.get_GEO(geo = serie, destdir = metadata_dir)
logger.info(f'Descarga de metadata para las series de la plataforma {platform} finalizada')

# Descarga de archivos sra y transformacion a fastq.gz

In [ ]:
if not isLinux: #( Windows ).
    # Cambiar directorio principal a las herramientas de SRAToolKit.
    os.chdir(sratoolkit_dir)
    logger.info(f'Cambio de directorio: {sratoolkit_dir}')

In [ ]:
# Descarga con sra toolkit
db = SRAweb()
# Ejecutar hasta que no haya errores de descarga o se considere finalizado.
logger.info('Descarga de archvios sra y transformacion a fastq.gz')
tryAgain = 'si'
while (tryAgain == 'si'):
    errores = {}
    countSerie = [0,len(series.keys())]
    for serie in series.keys():
        countSerie[0] += 1
        print(f'{"*"*(len(serie)+14)}\n****** {serie} ****** [{countSerie[0]}/{countSerie[1]}] \n{"*"*(len(serie)+14)}\n')
        # Control de errores de descarga.
        errores[serie] = []
        # Filtra las muestras para que sean solo de la plataforma elegida.
        gsmsKeys = [x for x in series[serie].gsms.keys() if series[serie].gsms[x].metadata['platform_id'][0]==platform]
        countMuestra = [0,len(gsmsKeys)]
        for muestra in gsmsKeys:
            countMuestra[0] += 1
            print(f'****** {muestra} ****** [{countMuestra[0]}/{countMuestra[1]}]')
            path = f'{experiments_dir}/{serie}'
            if os.path.isfile(f'{path}/{muestra}.fastq.gz'):
                print(f'El archivo {muestra}.fastq.gz ya existe\n')
            else:
                has_error = True
                num_error = 0
                while (has_error and num_error < 10):
                    try:
                        # Obtiene la metadata con los id SRR.
                        sampleMetadata = db.sra_metadata(muestra)
                        pathTemp = f'{experiments_dir}/{serie}/{muestra}'
                        os.makedirs(pathTemp) if not os.path.exists(pathTemp) else None # Crea la carpeta si no existe.
                        for srr in sampleMetadata['run_accession']:
                            logger.info(f'Descarga iniciada: {srr}')
                            # Descargar el archivo sra.
                            call([ 'prefetch', '-p', '1', '-O', pathTemp, srr ], shell = True)
                            filePath = f'{pathTemp}/{srr}.sra' # Nombre del archvivo sra descargado.
                            shutil.move(f'{pathTemp}/{srr}/{srr}.sra', filePath) # Mover el archivo descargado a la carpeta adecuada.
                            logger.info(f'Descarga finalizada: {srr}')
                            # Eliminar la carpeta vacia.
                            os.rmdir(f'{pathTemp}/{srr}')
                            # Transformacion a fastq.gz.
                            logger.info('Transformación a fastq.gz en proceso.')
                            call(['fastq-dump','--outdir',pathTemp,'--gzip','--skip-technical','--readids','--read-filter','pass','--dumpbase','--split-3','--clip',filePath],shell=True) 
                            os.remove(filePath) # Borra el archvio sra despues de ser transformado a fastq.
                            logger.info('Transformacion a fastq.gz finalizada.')
                        fastqFiles = [f'{pathTemp}/{x}' for x in sorted(os.listdir(pathTemp))]
                        # mover y renombrar el(los) archivo(s) fastq a su respectiva carpeta de experimento.
                        index = 0
                        for file in fastqFiles:
                            if index == 0:
                                shutil.move(file,f'{path}/{muestra}.fastq.gz')
                                index += 1
                            else:
                                if (file.endswith('pass_2.fastq.gz')):
                                    shutil.move(file,f'{path}/{muestra}{f"_{index+1}" if index>1 else ""}_paired.fastq.gz')
                                else:
                                    shutil.move(file,f'{path}/{muestra}_{index+1}.fastq.gz')
                                    index += 1
                        # Eliminar la carpeta vacia.
                        os.rmdir(pathTemp)
                        has_error = False
                    except Exception as e:
                        logger.error(f'{e}.  Intento {num_error+1}')
                        has_error = True
                        num_error += 1
                if has_error:
                    logger.error(f'Ocurrio un error de descarga para la muestra {muestra} :c')
                    errores[serie].append(muestra)

    # ERRORES.
    for serie in errores.keys():
        if len(errores[serie]) > 0:
            logger.warning(f'La serie {serie} tuvo {len(errores[serie])} errores: {errores[serie]}')
        else:
            logger.info(f'La serie {serie} tuvo {len(errores[serie])} errores: {errores[serie]}')
    # Si hay errores de descarga se pregunta si desea repetir las descargas.
    if sum([len(errores[x]) for x in errores.keys()]) > 0:
        tryAgain = str(input('¿Desea ejecutar nuevamente? Teclea "Si" o "No": ')).lower()
    else:
        tryAgain = 'no'
logger.info('Descarga de archvios sra y transformacion a fastq.gz finalizada')

In [ ]:
# # Descarga usando Geoparse
# # Ejecutar hasta que no haya errores de descarga o se considere finalizado.
# logger.info('Descarga de archvios sra y transformacion a fastq.gz')
# tryAgain = 'si'
# while (tryAgain == 'si'):
#     errores = {}
#     countSerie = [0,len(series.keys())]
#     for serie in series.keys():
#         countSerie[0] += 1
#         print(f'{"*"*(len(serie)+14)}\n****** {serie} ****** [{countSerie[0]}/{countSerie[1]}] \n{"*"*(len(serie)+14)}\n')
#         # Control de errores de descarga.
#         errores[serie] = []
#         # Filtra las muestras para que sean solo de la plataforma elegida.
#         gsmsKeys = [x for x in series[serie].gsms.keys() if series[serie].gsms[x].metadata['platform_id'][0]==platform]
#         countMuestra = [0,len(gsmsKeys)]
#         for muestra in gsmsKeys:
#             countMuestra[0] += 1
#             print(f'****** {muestra} ****** [{countMuestra[0]}/{countMuestra[1]}]')
#             path = f'{experiments_dir}/{serie}'
#             if os.path.isfile(f'{path}/{muestra}.fastq.gz'):
#                 print(f'El archivo {muestra}.fastq.gz ya existe\n')
#             else:
#                 has_error = True
#                 num_error = 0
#                 while (has_error and num_error < 10):
#                     try:
#                         fastqName = series[serie].gsms[muestra].download_SRA(email=correo,directory=path,filetype="fastq",keep_sra=False) 
#                         fastqName['SRA'].sort()
#                         # mover y renombrar el(los) archivo(s) fastq a su respectiva carpeta de experimento.
#                         index = 0
#                         for file in fastqName['SRA']:
#                             if index == 0:
#                                 shutil.move(file,f'{path}/{muestra}.fastq.gz')
#                                 index += 1
#                             else:
#                                 if (file.endswith('pass_2.fastq.gz')):
#                                     shutil.move(file,f'{path}/{muestra}{f"_{index+1}" if index>1 else ""}_paired.fastq.gz')
#                                 else:
#                                     shutil.move(file,f'{path}/{muestra}_{index+1}.fastq.gz')
#                                     index += 1
#                         # Eliminar la carpeta vacia.
#                         if isLinux:
#                             os.rmdir('/'.join(fastqName['SRA'][0].split('/')[:-1]))
#                         else:
#                             os.rmdir('/'.join(fastqName['SRA'][0].split('\\')[:-1]))
#                         has_error = False
#                     except:
#                         logger.error(f'Intento {num_error+1}')
#                         has_error = True
#                         num_error += 1
#                 if has_error:
#                     logger.error(f'Ocurrio un error de descarga para la muestra {muestra} :c')
#                     errores[serie].append(muestra)

#     # ERRORES.
#     for serie in errores.keys():
#         if len(errores[serie]) > 0:
#             logger.warning(f'La serie {serie} tuvo {len(errores[serie])} errores: {errores[serie]}')
#         else:
#             logger.info(f'La serie {serie} tuvo {len(errores[serie])} errores: {errores[serie]}')
#     # Si hay errores de descarga se pregunta si desea repetir las descargas.
#     if sum([len(errores[x]) for x in errores.keys()]) > 0:
#         tryAgain = str(input('¿Desea ejecutar nuevamente? Teclea "Si" o "No": ')).lower()
#     else:
#         tryAgain = 'no'
# logger.info('Descarga de archvios sra y transformacion a fastq.gz finalizada')

# Conteo de genes con ayuda de un script en R

In [ ]:
def borrarArchvivosDeEjecucionesAnteriores(path):
    '''Borra los archivos residuales de ejecuciones anteriores del conteo de genes
    Parameters
    ----------
    path : string
        Dirección de la carpeta de una serie con los archvios residuales de ejecuciones de conteo de genes anteriores.  
    Returns
    -------
    void
    '''
    if (os.path.isdir(f'{path}/bam')):
        shutil.rmtree(f'{path}/bam')
    if (os.path.isfile(f'{path}/rnaFeatureCount.rds')):
        os.remove(f'{path}/rnaFeatureCount.rds')
    if (os.path.isfile(f'{path}/rnaFeatureCount_paired.rds')):
        os.remove(f'{path}/rnaFeatureCount_paired.rds')
    for dire in os.listdir(path):
        if (dire.startswith('my_index.')):
            os.remove(f'{path}/{dire}')
        if (dire.endswith('_trimed.fastq.gz')):
            os.remove(f'{path}/{dire}')
        if (re.fullmatch('[0-9]+\.txt|[0-9]+-[0-9]+\.txt',dire)):
            os.remove(f'{path}/{dire}')
    logger.info(f'Carpeta {path} limpiada correctamente')

In [ ]:
logger.info(f'Conteo de genes para las muestras de cada serie de la plataforma {platform}')
# Expresion regular para conseguir el id GEO de archivos fastq.gz.
re_fastqfile = re.compile('(GSM[0-9]*)\.fastq\.gz')
# Conteo de genes para todas las series descargadas.
countSerie = [0, len(series.keys())]
for serie in series.keys():
    countSerie[0] += 1
    print(f'{"*"*(len(serie)+14)}\n****** {serie} ****** [{countSerie[0]}/{countSerie[1]}] \n{"*"*(len(serie)+14)}\n')
    path = f'{experiments_dir}/{serie}'
    # Filtra las muestras para que sean solo de la plataforma elegida.
    gsmsKeys = [x for x in series[serie].gsms.keys() if series[serie].gsms[x].metadata['platform_id'][0]==platform] 
    try:
        os.chdir(path)
        response = 'si'
        if (os.path.isfile(f'{path}/gene_counts.csv')):
            print(f'Ya existe un conteo de genes para la serie {serie}')
            response = str(input('¿Aún asi esea realizar un nuveo conteo? Teclea "Si" o "No": '))
        inexistFastqFiles = [file for file in gsmsKeys if file not in re_fastqfile.findall(" ".join(os.listdir(path)))]
        if (len(inexistFastqFiles) > 0 and response.lower() == 'si'):
            print(f'\nEn la carpeta de la serie {serie} faltan {len(inexistFastqFiles)} archivos fastq.gz de las muestras: {", ".join(inexistFastqFiles)}')
            response = str(input('¿Aún asi desea continuar con el conteo de genes de las muestras existentes? Teclea "Si" o "No": '))
        if (response.lower() == 'si'):
            logger.info(f'Ejecutando script para la serie {serie}')
            borrarArchvivosDeEjecucionesAnteriores(path)
            # Crear un txt con las muestas que hay menos las que faltan.
            with open(f'{len(gsmsKeys)}-{len(inexistFastqFiles)}.txt' if len(inexistFastqFiles)>0 else f'{len(gsmsKeys)}.txt' ,'w') as handle: 
                for sample in inexistFastqFiles:
                    handle.write(f'{sample}\n')
                logger.info(f'Documento de texto con las muestras descartadas creado correctamente ({handle.name})')
            # Ejecucion del script de R.
            if isLinux:# Función call en linux.
                print(call([rscript_dir,genecount_script,path,genomaref_fnafile,genomaannot_gtffile]))
            else:
                !{rscript_dir} {genecount_script} {path} {genomaref_fnafile} {genomaannot_gtffile}
        else:
            logger.info(f'Se ha cancelado el conteo de genes para la serie {serie}')
    except FileNotFoundError:
        logger.error(f'No existe la carpeta {path}')
logger.info('Conteo de genes finalizado')

# Carga de datos

## Conteo de genes (gene_counts)

In [ ]:
os.chdir(work_dir)
logger.info(f'Cambio de directorio: {work_dir}')

In [ ]:
logger.info(f'INICIO. Tratamiento del conteo de genes para la plaaforma {platform}')
# Expresión regular para filtrar los nombres de las columnas del csv generado por el script de R.
re_GSM = re.compile('GSM[0-9]*_[0-9]*|GSM[0-9]*')
counts = pd.DataFrame(columns=series.keys(),index=['counts','CPM','RPKM','TMP','TMM'])
# Lectura de los archivos csv de todas las series.
for serie in counts.keys():
    path = f'{experiments_dir}/{serie}'
    if os.path.isfile(f'{path}/gene_counts.csv'):
        counts[serie]['counts'] = pd.read_csv(f'{path}/gene_counts.csv',index_col=0)
        counts[serie]['counts'].columns = re_GSM.findall("".join(counts[serie]['counts'].columns))
        # Remplaza valores nulos con ceros.
        counts[serie]['counts'].fillna(0,inplace=True)
    else:
        logger.error(f'No existe el archivio gene_counts.csv para la serie {serie}')
logger.info('Lectura de archivos (gene_counts.csv) finalizado')

## Longitud de genes

In [ ]:
# Lectura del archivo de anotaciones del genoma (gtf).
length_genes = read_gtf(genomaannot_gtffile)
# Filtra solo los genes.
length_genes = (length_genes[length_genes['feature']=='gene'])[['gene_id','start','end']]
# Obtiene la longitud de los genes.
length_genes['length'] = length_genes['end'] - length_genes['start']
length_genes = length_genes.set_index('gene_id')['length']
logger.info('Lectura de longitud de genes finalizado')

# Normalizaciones

## Normalización CPM

In [ ]:
for index,serie in enumerate(counts.loc['counts'].dropna().keys()):
    nmCPM = norm()
    nmCPM.cpm(df = counts[serie]['counts'])
    counts[serie]['CPM'] = nmCPM.cpm_norm
logger.info('Normalización CPM finalizada')

## Normalización RPKM

In [ ]:
for serie in counts.loc['counts'].dropna().keys():
    nmRPKM = norm()
    nmRPKM.rpkm(df = counts[serie]['counts'].merge(length_genes,left_index=True,right_index=True),gl = 'length') 
    counts[serie]['RPKM'] = nmRPKM.rpkm_norm
logger.info('Normalización RPKM finalizada')

## Normalización TMP

In [ ]:
for serie in counts.loc['counts'].dropna().keys():
    nmTMP = norm()
    nmTMP.tpm(df = counts[serie]['counts'].merge(length_genes,left_index=True,right_index=True),gl = 'length')
    counts[serie]['TMP'] = nmTMP.tpm_norm
logger.info('Normalización TMP finalizada')

## Normalización TMM

In [ ]:
#--------------------------------------
#  edgeR TMM normalization
#--------------------------------------
def edger_calcnormfactors(counts_df, ref=None, logratio_trim=0.3, sum_trim=0.05, acutoff=-1e10, verbose=False):
    # Author: Francois Aguet
    # https://github.com/broadinstitute/pyqtl/blob/master/qtl/norm.py
    """
    Calculate TMM (Trimmed Mean of M values) normalization.
    Reproduces edgeR::calcNormFactors.default
    Scaling factors for the library sizes that minimize
    the log-fold changes between the samples for most genes.
    Effective library size: TMM scaling factor * library size
    References:
     [1] Robinson & Oshlack, 2010
     [2] R functions:
          edgeR::calcNormFactors.default
          edgeR:::.calcFactorWeighted
          edgeR:::.calcFactorQuantile
    """

    # discard genes with all-zero counts
    Y = counts_df.values.copy()
    allzero = np.sum(Y>0,axis=1)==0
    if np.any(allzero):
        Y = Y[~allzero,:]

    # select reference sample
    if ref is None:  # reference sample index
        f75 = np.percentile(Y/np.sum(Y,axis=0), 75, axis=0)
        ref = np.argmin(np.abs(f75-np.mean(f75)))
        if verbose:
            print('Reference sample index: '+str(ref))

    N = np.sum(Y, axis=0)  # total reads in each library

    # with np.errstate(divide='ignore'):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        # log fold change; Mg in [1]
        logR = np.log2((Y/N).T / (Y[:,ref]/N[ref])).T
        # average log relative expression; Ag in [1]
        absE = 0.5*(np.log2(Y/N).T + np.log2(Y[:,ref]/N[ref])).T
        v = (N-Y)/N/Y
        v = (v.T + v[:,ref]).T  # w in [1]

    ns = Y.shape[1]
    tmm = np.zeros(ns)
    for i in range(ns):
        fin = np.isfinite(logR[:,i]) & np.isfinite(absE[:,i]) & (absE[:,i] > acutoff)
        n = np.sum(fin)

        loL = np.floor(n*logratio_trim)+1
        hiL = n + 1 - loL
        loS = np.floor(n*sum_trim)+1
        hiS = n + 1 - loS
        rankR = stats.rankdata(logR[fin,i])
        rankE = stats.rankdata(absE[fin,i])
        keep = (rankR >= loL) & (rankR <= hiL) & (rankE >= loS) & (rankE <= hiS)
        # in [1], w erroneously defined as 1/v ?
        tmm[i] = 2**(np.nansum(logR[fin,i][keep]/v[fin,i][keep]) / np.nansum(1/v[fin,i][keep]))

    tmm = tmm / np.exp(np.mean(np.log(tmm)))
    return tmm


def edger_cpm_default(counts_df, lib_size=None, log=False, prior_count=0.25):
    """
    edgeR normalized counts
    Reproduces edgeR::cpm.default
    """
    if lib_size is None:
        lib_size = counts_df.sum(axis=0)
    if log:
        prior_count_scaled = lib_size/np.mean(lib_size) * prior_count
        lib_size <- lib_size + 2 * prior_count_scaled
    lib_size = 1e-6 * lib_size
    if log:
        return np.log2((counts_df + prior_count_scaled)/lib.size)
    else:
        return counts_df / lib_size


def edger_cpm(counts_df, tmm=None, normalized_lib_sizes=True):
    """
    Return edgeR normalized/rescaled CPM (counts per million)
    Reproduces edgeR::cpm.DGEList
    """
    lib_size = counts_df.sum(axis=0)
    if normalized_lib_sizes:
        if tmm is None:
            tmm = edger_calcnormfactors(counts_df)
        lib_size = lib_size * tmm
    return counts_df / lib_size * 1e6
#--------------------------------------
#  edgeR TMM normalization
#--------------------------------------

In [ ]:
for serie in counts.loc['counts'].dropna().keys():
    counts[serie]['TMM'] = edger_cpm(counts_df = counts[serie]['counts'])
logger.info('Normalización TMM finalizada')
logger.info(f'FIN. Tratamiento del conteo de genes para la plaaforma {platform}')

# Visualización de datos

In [ ]:
def get_maxUpperWhisker( df ):
    '''Obtener el valor maximo entre todos los brazos superiores de los boxplot generados por un data frame.
    Parameters
    ----------
    df : DataFrame
        Dataframe de pandas con columnas-muestras, filas-genes, values-float32.
    Returns
    -------
    Float
        Valor maximo entre todos los brazos superiores de los boxplot
    '''
    describe = df.astype('float32').describe()
    return max(describe.loc['75%'] + (1.5*(describe.loc['75%'] - describe.loc['25%'])))
def get_minLowerWhisker( df ):
    '''Obtener el valor minimo entre todos los brazos inferirores de los boxplot de un data frame
    Parameters
    ----------
    df : DataFrame
        Dataframe de pandas con columnas-muestras, filas-genes, values-float32.
    Returns
    -------
    Float
        Valor minimo entre todos los brazos inferiores de los boxplot.
    '''
    describe = df.describe()
    lowerWhisker = min(describe.loc['25%'] - (1.5*(describe.loc['75%'] - describe.loc['25%'])))
    lowerVal = df.min().min()
    return lowerWhisker if lowerVal < lowerWhisker else lowerVal

def graficarBoxplot(df, title = 'Sin título', showfliers = False):
    '''Gráfica los diagramas de caja y brazos de un data frame.
    Parameters
    ----------
    df : DataFrame
        Dataframe de pandas con columnas-muestras, filas-genes, values-float32.
    title : String
        Título de la gráfica.
    showfliers : Boolean
        Indica si se quiere mostrar los valores atipicos de las diagrams de caja y brazos.
    Returns
    -------
    Float
        Valor minimo entre todos los brazos inferiores de los boxplot.
    '''
    plot = df.boxplot(column = df.columns.to_list(), return_type = 'axes',showfliers = showfliers)
    plot.set_ylim(get_minLowerWhisker(df),get_maxUpperWhisker(df))
    plot.set_xticklabels([])
    plot.set_title( title )
    plot.tick_params(axis = 'y', colors = '0.9') # 0.9 light gray
    plot.tick_params(axis = 'x', colors = 'none')
    plot.title.set_color('0.9')
    plot.xaxis.grid()
    return plt.show()

In [ ]:
if showGraphs:
    logger.info('Visualización de boxplots')
    if (counts.isna().sum().sum() > 0):
        logger.warning(f'Las series {", ".join([x for x in counts.columns if counts[x].isna().sum() > 0])} no tienen una grafica asignada.') 
    countSerie = [0, len(counts.dropna(axis=1).columns)]
    for serie in counts.dropna(axis=1).columns:
        countSerie[0] += 1
        print(f'****** {serie} ****** [{countSerie[0]}/{countSerie[1]}]')
        graficarBoxplot(counts[serie]['counts'], title=f'Conteo de genes {serie}')
        graficarBoxplot(counts[serie]['CPM'], title=f'Normalización CPM {serie}')
        graficarBoxplot(counts[serie]['RPKM'], title=f'Normalización RPKM {serie}')
        graficarBoxplot(counts[serie]['TMP'], title=f'Normalización TMP {serie}')
        graficarBoxplot(counts[serie]['TMM'], title=f'Normalización TMM {serie}')

# Creación de documentos 

## Gráficas

In [ ]:
logger.info('INICIO. Creación de documentos')
os.makedirs(documents_dir) if not os.path.exists(documents_dir) else None # Crea la carpeta si no existe
os.chdir(documents_dir)
logger.info(f'Cambio de directorio {documents_dir}')
def get_bytesPdfBoxplotImage(serie, kind, boxpoints, df):
    '''Crea un pdf en bytes con la imagen del diagrama de caja y brazos de un DataFrame.
    Parameters
    ----------
    serie : String
        Nombre de la serie (experimento).
    kind : String
        Tipo de datos (counts, CPM, RPKM, TMP, TMM).
    boxpoints : Boolean
        Indica si se quieren mostrar los valores atipicos de los diagramas de caja y brazos.
    df : DataFrame
        Dataframe de pandas con: columnas-muestras, filas-genes, values-float32.
    Returns
    -------
    Bytes
        Pdf en bytes de la imagen del boxplot.
    '''
    df = df.astype('float32')
    title = f'Conteo de genes {serie}' if kind == 'counts' else f'Normalización {kind} {serie}'
    fig = df.iplot(kind='box',boxpoints=boxpoints,title=title,yrange=[0,get_maxUpperWhisker(df)],asFigure=True) 
    fig.update_traces(marker_opacity = 0.5, selector=dict(type='box'))
    fig.update_traces(marker_size = 2, selector=dict(type='box'))
    fig.update_layout(showlegend = False)
    fig.update_layout(font_size = 10)
    fig.update_layout(width = (700 + len(df.columns)) if len(df.columns)>100 else 700)
    fig.update_layout(height = 450)
    return fig.to_image(format = 'pdf')

def generatePdf(serie, boxpoints = False, df = counts):
    '''Genera un archivo pdf con todas las gráficas de una serie
    Parameters
    ----------
    serie : String
        Nombre de la serie (experimento).
    boxpoints : Boolean
        Indica si se quieren mostrar los valores atipicos de los diagramas de caja y brazos.
    df : DataFrame
        Dataframe de pandas con: columnas-series, filas-kind(counts, CPM, RPKM, TMP, TMM), values-DataFrame.
    Returns
    -------
    Void
    '''
    # Juntar los plotly en un pdf
    merger = PdfFileMerger()
    for kind in df.index:
        merger.append(io.BytesIO(get_bytesPdfBoxplotImage(serie,kind,boxpoints,df[serie][kind])))
    with open(f'{serie}.pdf','wb') as handle:
        merger.write(handle) 
    logger.info(f'PDF generado correctamente: {serie}.pdf')

In [ ]:
if (counts.isna().sum().sum() > 0):
    logger.warning(f'Las series {", ".join([x for x in counts.columns if counts[x].isna().sum() > 0])} no tienen datos asignados.') 
countSerie = [0, len(counts.dropna(axis=1).columns)]
for serie in counts.dropna(axis = 1).columns:
    countSerie[0] += 1
    logger.info(f'{serie}  [{countSerie[0]}/{countSerie[1]}]')
    generatePdf(serie,'outliers',counts)
logger.info('Pdfs de boxplots generados correctamente')

## Metadata de las series

In [ ]:
df_metadata = pd.DataFrame([],columns=['geo_accession','title','summary','status','submission_date','last_update_date','type','overall_design','contributor','contact_email','contact_institute','contact_laboratory','contact_country','platform_id','total_samples','num_samples_discarded','samples_discarded']) 
# Solo guarda las series que fueron procesadas
for serie in counts.dropna(axis=1).columns:
    newRow = {}
    for title in [x for x in df_metadata.columns.to_list() if x not in ['total_samples','num_samples_discarded','samples_discarded']]:
        try:
            newRow[title] = ', '.join(series[serie].metadata[title]) if title != 'contributor' else ', '.join(list(map(lambda x: str(x).replace(',',' '),series[serie].metadata[title])))
        except:
            newRow[title] = None
    # Filtra las muestras para que sean solo de la plataforma elegida
    gsmsKeys = [x for x in series[serie].gsms.keys() if series[serie].gsms[x].metadata['platform_id'][0]==platform]
    newRow["total_samples"] = len(gsmsKeys)
    newRow["num_samples_discarded"] = None
    newRow["samples_discarded"] = None
    newRow["platform_id"] = platform

    # Registro de muestras descartadas
    discarded = [x for x in gsmsKeys if x not in counts[serie]['counts'].columns]
    if len(discarded) == 0:
        newRow["num_samples_discarded"] = 0
        newRow["samples_discarded"] = 'No samples were discarded'
    else:
        newRow["num_samples_discarded"] = len(discarded)
        newRow["samples_discarded"] = ', '.join(discarded)

    df_metadata = df_metadata.append(newRow, ignore_index=True)
    
df_metadata.set_index('geo_accession',inplace=True)
df_metadata.to_csv(f'{documents_dir}/experiments_data-{platform}.csv')
logger.info(f'Metadata de series generado correctamente en: {documents_dir}/experiments_data-{platform}.csv')

## Metadata de las muestras

In [ ]:
df_metadataSample = pd.DataFrame([],columns=['geo_accession','title','organism_ch1','status','submission_date','last_update_date','growth_protocol_ch1','molecule_ch1','extract_protocol_ch1','data_processing','data_processing','instrument_model','library_selection','library_source','library_strategy','platform_id']) 
# Solo guarda las series que fueron procesadas
for serie in counts.dropna(axis=1).columns:
    # Filtra las muestras para que sean solo de la plataforma elegida
    gsmsKeys = [x for x in series[serie].gsms.keys() if series[serie].gsms[x].metadata['platform_id'][0]==platform]
    # Registro de muestras descartadas
    discarded = [x for x in gsmsKeys if x not in counts[serie]['counts'].columns]
    for muestra in [x for x in gsmsKeys if x not in discarded]: # Si hay muestras descartadas no se añaden a este archivo
        newRow = {}
        for title in df_metadataSample.columns:
            try:
                newRow[title] = f'\n'.join(series[serie].gsms[muestra].metadata[title])
            except:
                newRow[title] = None

        df_metadataSample = df_metadataSample.append(newRow, ignore_index=True)
    
df_metadataSample.set_index('geo_accession',inplace=True)
df_metadataSample.to_csv(f'{documents_dir}/samples_data-{platform}.csv')
logger.info(f'Documento generado correctamente en: {documents_dir}/samples_data-{platform}.csv')
logger.info('FIN. Creación de documentos')

# Creación de la matriz de muestras

In [ ]:
logger.info('INICIO. Creación de matrices de las muestras')
genes_id = read_gtf(genomaannot_gtffile) 
genes_id = genes_id[genes_id['feature']=='gene'].reset_index()['gene_id']
logger.info(f'Lectura de genes del archivo {genomaannot_gtffile} finalizado')
samples = pd.DataFrame(columns=['beforeBatchN','afterBatchN'],index=['CPM','RPKM','TMP','TMM'])
for kind in samples.index:
    logger.info(f'Inicio. Normalización de batch a {kind}')
    batchVector = []
    temp = pd.DataFrame(index=genes_id)
    for index,serie in enumerate(counts.dropna(axis = 1).columns):
        temp = temp.merge(counts[serie][kind],left_index=True,right_index=True,how='left').fillna(0)
        batchVector += [index for x in range(len(counts[serie][kind].columns))] 
        
    #### REMOVER LOS GENES QUE TENGAN VARIANZA IGUAL A CERO
    drpgen = pd.DataFrame([],columns = temp.columns)
    for gen in temp.index:
        if temp.loc[gen].var() == 0:
            drpgen = drpgen.append(temp.loc[gen])
            temp = temp.drop([gen],axis=0)
    logger.info(f'{len(drpgen.index)} genes tienen varianza igual a cero')
    
    samples['beforeBatchN'][kind] = temp.append(drpgen).astype('float32')
    # Normalizacion de batch
    try:
        samples['afterBatchN'][kind] = pycombat(temp.astype('float32'),batchVector).append(drpgen).astype('float32')
        logger.info(f'Normalización de Batch aplicada correctamente a {kind}')
    except:
        samples['afterBatchN'][kind] = np.NaN
        logger.error(f'Ocurrio un problema en la normalización de batch de {kind}')
logger.info('FIN. Creación de matrices de las muestras')

In [ ]:
if showGraphs:
    logger.info('Visualización de boxplots')
    plt.rcParams["figure.figsize"] = (30,15)
    for y in samples.index:
        for x in samples.columns:
            print(f'{x} tipo {y}')
            graficarBoxplot(samples[x][y],title=f'{x} tipo {y}')

In [ ]:
# Genera archivos csv para cada uno de las matrices (CPM, RPKM, TMP, TMM) a las que se le aplico correctamente el ajuste de batch 
for kind in samples.dropna(axis = 0).index:
    samples['afterBatchN'][kind].to_csv(f'{documents_dir}/matrizAfterBatch{kind}.csv')
    logger.info(f'Matriz guardada en: {documents_dir}/matrizAfterBatch{kind}.csv')